In [9]:
import os
%pwd

'g:\\MLOps_Project\\MLOps_Project\\research'

In [10]:
os.chdir("../")

In [11]:
%pwd

'g:\\MLOps_Project\\MLOps_Project'

In [12]:
import dagshub
dagshub.init(repo_owner='Adarsh63', repo_name='MLOps_Project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

[2025-09-07 13:11:54,735: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as Adarsh63

[2025-09-07 13:11:54,763: INFO: helpers: Accessing as Adarsh63]
[2025-09-07 13:11:55,324: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/Adarsh63/MLOps_Project "HTTP/1.1 200 OK"]
[2025-09-07 13:11:55,839: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "Adarsh63/MLOps_Project"

[2025-09-07 13:11:55,846: INFO: helpers: Initialized MLflow to track repo "Adarsh63/MLOps_Project"]


Repository Adarsh63/MLOps_Project initialized!

[2025-09-07 13:11:55,857: INFO: helpers: Repository Adarsh63/MLOps_Project initialized!]
🏃 View run unique-crab-205 at: https://dagshub.com/Adarsh63/MLOps_Project.mlflow/#/experiments/0/runs/28eaa662469c46e69adda4360a0c6017
🧪 View experiment at: https://dagshub.com/Adarsh63/MLOps_Project.mlflow/#/experiments/0


In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Adarsh63/MLOps_Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Adarsh63"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b82416e02e08e727dd58201b443b8b9d051ec776"

In [2]:
pip install mlflow==2.22.2

In [20]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [21]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.Salary,
            mlflow_uri="https://dagshub.com/Adarsh63/MLOps_Project.mlflow",
           
        )

        return model_evaluation_config


In [23]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [24]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [25]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-09-07 13:17:11,445: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-07 13:17:11,451: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-07 13:17:11,458: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-07 13:17:11,461: INFO: common: created directory at: artifacts]
[2025-09-07 13:17:11,466: INFO: common: created directory at: artifacts/model_evaluation]
[2025-09-07 13:17:12,114: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/09/07 13:17:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/09/07 13:17:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.


🏃 View run gentle-gnu-996 at: https://dagshub.com/Adarsh63/MLOps_Project.mlflow/#/experiments/0/runs/36865ff11f32447d941cebaa2a14334a
🧪 View experiment at: https://dagshub.com/Adarsh63/MLOps_Project.mlflow/#/experiments/0
